In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as ols
import statsmodels.api as sm

In [2]:
returns = pd.read_csv("crsp data.csv")
returns.head()

C:\Users\John Flume\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,PERMNO,date,SHRCD,SICCD,SHRCLS,PRC,RET,SHROUT
0,10000,12/31/1985,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,01/31/1986,10.0,3990,A,-4.3750,C,3680.0
2,10000,02/28/1986,10.0,3990,A,-3.2500,-0.257143,3680.0
3,10000,03/31/1986,10.0,3990,A,-4.4375,0.365385,3680.0
4,10000,04/30/1986,10.0,3990,A,-4.0000,-0.098592,3793.0


In [3]:
crsp_comp = pd.read_csv('crsp comp m.csv')
crsp_comp.head()

,GVKEY,LPERMNO,datadate,fyear,indfmt,consol,popsrc,datafmt,curcd,csho,xrd,exchg,costat,sich,prcc_c,prcc_f,sic
0,1000,25881,12/31/1970,1970,INDL,C,D,STD,USD,2.446,NaN,12,I,NaN,10.000,10.000,3089.0
1,1000,25881,12/31/1971,1971,INDL,C,D,STD,USD,2.995,NaN,12,I,NaN,5.750,5.750,3089.0
2,1000,25881,12/31/1972,1972,INDL,C,D,STD,USD,2.902,NaN,12,I,NaN,5.125,5.125,3089.0
3,1000,25881,12/31/1973,1973,INDL,C,D,STD,USD,2.840,NaN,12,I,NaN,1.750,1.750,3089.0
4,1000,25881,12/31/1974,1974,INDL,C,D,STD,USD,2.150,NaN,12,I,NaN,2.125,2.125,3089.0


In [4]:
fama = pd.read_csv("F-F_Research_Data_Factors.csv")

In [5]:
fama = fama.rename(columns = {'Unnamed: 0': 'date'})
fama.head()

,date,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.38,-2.73,0.22
1,192608,2.64,-1.47,4.14,0.25
2,192609,0.36,-1.39,0.12,0.23
3,192610,-3.24,-0.13,0.65,0.32
4,192611,2.53,-0.16,-0.38,0.31


####  Fama French DF Cleaning

In [6]:
# Creating a year column from the date
fama['year'] = fama['date'].apply(lambda x : str(x)[:4])

# Creating a month column from the date
fama['month'] = fama['date'].apply(lambda x : str(x)[-2:])

# Converting date to datetime
fama['date'] = pd.to_datetime(fama['year'] + '-' + fama['month']).dt.strftime('%Y-%m')

# Removing the Year and Month
fama = fama.drop(columns = ['year', 'month'])

# Converting rf to decimals rather than %
fama['RF'] = fama['RF'] / 100

# Converting mkt prem to decimals rather than %
fama['Mkt-RF'] = fama['Mkt-RF'] / 100

#### CRSP / Compustat Merged DF Cleaning

In [7]:
# Fill NaN for R&D Expense to 0 and fill SIC Codes that are missing to the historical SIC
crsp_comp['xrd'] = crsp_comp['xrd'].fillna(0)
crsp_comp['sic'] = crsp_comp['sic'].fillna(crsp_comp['sich'])

# Filter for U.S. Exchanges and Remove Financial Companies
crsp_comp = crsp_comp[crsp_comp['exchg'].between(11,19)]
crsp_comp = crsp_comp[~crsp_comp['sic'].between(6011, 6999)]

# Shift R&D to create time period R&D expense & Shift LPERMNO to match the R&D Expense
crsp_comp['xrd 1'] = crsp_comp['xrd'].shift(1)
crsp_comp['xrd 2'] = crsp_comp['xrd'].shift(2)
crsp_comp['xrd 3'] = crsp_comp['xrd'].shift(3)
crsp_comp['xrd 4'] = crsp_comp['xrd'].shift(4)

crsp_comp['LPERMNO 1'] = crsp_comp['LPERMNO'].shift(1)
crsp_comp['LPERMNO 2'] = crsp_comp['LPERMNO'].shift(2)
crsp_comp['LPERMNO 3'] = crsp_comp['LPERMNO'].shift(3)
crsp_comp['LPERMNO 4'] = crsp_comp['LPERMNO'].shift(4)

crsp_comp = crsp_comp[(crsp_comp['LPERMNO'] == crsp_comp['LPERMNO 1']) & 
                                     (crsp_comp['LPERMNO'] == crsp_comp['LPERMNO 2']) &
                                      (crsp_comp['LPERMNO'] == crsp_comp['LPERMNO 3']) &
                                      (crsp_comp['LPERMNO'] == crsp_comp['LPERMNO 4'])]

# Calculate total xrd
crsp_comp['total xrd'] = (crsp_comp['xrd'] + 0.8 * crsp_comp['xrd 1'] + 
                                        0.6 * crsp_comp['xrd 2'] + 0.4 * crsp_comp['xrd 3'] + 
                                        0.2 * crsp_comp['xrd 4'])

# Shift R&D back one year because R&D info not been released yet to trade on
crsp_comp['year traded'] = crsp_comp['fyear'] + 1

# Keep necessary columns
crsp_comp = crsp_comp[['year traded', 'LPERMNO', 'xrd', 'total xrd', 'sic', 'csho']]

#### Stock Returns DF Cleaning

In [8]:
# Change to datetime object 
returns['date'] = pd.to_datetime(returns['date'])

# Create a year column
returns['year'] = returns['date'].dt.year

# Re-format the date
returns['date'] = returns['date'].dt.strftime('%Y-%m')

# Filter Class A shares or where Share Class is missing
returns = returns[(returns['SHRCLS'] == 'A') | (returns['SHRCLS'].isna())]

# Remove C & B from returns
returns = returns[pd.to_numeric(returns['RET'], errors = 'coerce').notna()]

# Convert returns to a float
returns['RET'] = returns['RET'].apply(lambda x : float(x))

# Keep necessary columns
returns = returns[['date', 'PERMNO', 'PRC', 'RET', 'year']]

#### Merging crsp_comp dataframe with the stock returns datafame

In [9]:
df = pd.merge(crsp_comp, returns, how = 'left', left_on = ['LPERMNO', 'year traded'], right_on = ['PERMNO', 'year'])
df.head()

,year traded,LPERMNO,xrd,total xrd,sic,csho,date,PERMNO,PRC,RET,year
0,1975,25881,0.0,0.0,3089.0,2.15,1975-01,25881.0,-2.1875,0.029412,1975.0
1,1975,25881,0.0,0.0,3089.0,2.15,1975-02,25881.0,2.6250,0.200000,1975.0
2,1975,25881,0.0,0.0,3089.0,2.15,1975-03,25881.0,3.0000,0.142857,1975.0
3,1975,25881,0.0,0.0,3089.0,2.15,1975-04,25881.0,-2.6875,-0.104167,1975.0
4,1975,25881,0.0,0.0,3089.0,2.15,1975-05,25881.0,-2.5625,-0.046512,1975.0


In [10]:
# Take absolute value of price due to ask bid spread issue
df['PRC'] = df['PRC'].apply(lambda x : abs(x))
 
# Calculating market value of equity
df['me'] = df['csho'].copy() * df['PRC'].copy()

# Calculating relative R&D
df['xrd_me'] = df['total xrd'].copy() / df['me'].copy()

# Shift returns 3 months to account for information lag
df['RET'] = df['RET'].shift(-3)

# Range of data after 1980 and before 2021
df = df[(df['year'] > 1980) & (df['year'] < 2021)]

# Create df for R&D and non-R&D firms
rd_df = df[df['xrd_me'] != 0]
no_rd = df[df['xrd_me'] == 0]

In [11]:
# Split into R&D Quintiles/Ranks
#xrd_only['Rank'] = xrd_only.groupby('year')['rdc_me'].transform(lambda x: pd.qcut(x, 5, labels = ['low', 2, 3, 4, 'high']))
rd_df['rank'] = rd_df.groupby(['year'])['xrd_me'].apply(lambda x : pd.qcut(x, 5, labels = range(1,6)))
rd_df.head()

<ipython-input-11-e34f76041159>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rd_df['rank'] = rd_df.groupby(['year'])['xrd_me'].apply(lambda x : pd.qcut(x, 5, labels = range(1,6)))


,year traded,LPERMNO,xrd,total xrd,sic,csho,date,PERMNO,PRC,RET,year,me,xrd_me,rank
841,1984,50906,5.223,15.0334,3661.0,5.728,1984-01,50906.0,13.750,-0.111111,1984.0,78.760,0.190876,4
842,1984,50906,5.223,15.0334,3661.0,5.728,1984-02,50906.0,13.375,0.000000,1984.0,76.612,0.196228,4
843,1984,50906,5.223,15.0334,3661.0,5.728,1984-03,50906.0,11.250,0.200000,1984.0,64.440,0.233293,5
844,1984,50906,5.223,15.0334,3661.0,5.728,1984-04,50906.0,10.000,0.000000,1984.0,57.280,0.262455,5
845,1984,50906,5.223,15.0334,3661.0,5.728,1984-05,50906.0,10.000,0.114583,1984.0,57.280,0.262455,5


# Task 1: Equal-Weighted Returns R&D Quintiles and Non-R&D firms

In [12]:
# Calculating Returns from 1981 to 2012 for both R&D and Non-R&D Firms
rd_81_12 = rd_df[(rd_df['date'] >= '1981-07') & (rd_df['date'] <= '2012-12')].groupby(['date', 'rank'])['RET'].mean().unstack(-1).dropna()
no_rd_81_12 = no_rd[(no_rd['date'] >= '1981-07') & (no_rd['date'] <= '2012-12')].groupby(['date'])['RET'].mean()

ret_81_12 = pd.merge(rd_81_12, no_rd_81_12, how = 'left', on = 'date')

# Calculating Returns from 1981.07 - 1999.12
rd_81_99 = rd_df[(rd_df['date'] >= '1981-07') & (rd_df['date'] <= '1999-12')].groupby(['date', 'rank'])['RET'].mean().unstack(-1).dropna()
no_rd_81_99 = no_rd[(no_rd['date'] >= '1981-07') & (no_rd['date'] <= '1999-12')].groupby(['date'])['RET'].mean()

ret_81_99 = pd.merge(rd_81_99, no_rd_81_99, how = 'left', on = 'date')

# Calculating Returns from 2000.01 - 2012.12
rd_00_12 = rd_df[(rd_df['date'] >= '2000-01') & (rd_df['date'] <= '2012-12')].groupby(['date', 'rank'])['RET'].mean().unstack(-1).dropna()
no_rd_00_12 = no_rd[(no_rd['date'] >= '2000-01') & (no_rd['date'] <= '2012-12')].groupby(['date'])['RET'].mean()

ret_00_12 = pd.merge(rd_00_12, no_rd_00_12, how = 'left', on = 'date')

In [13]:
pd.DataFrame(np.array([ret_81_12.mean(axis = 0) * 100, ret_81_99.mean(axis = 0) * 100, ret_00_12.mean(axis = 0) * 100]), columns = ['L', '2', '3', '4', 'H', 'Non R&D'], index = ['1981.07 - 2012.12', '1981.07 - 1999.12', '2000.01 - 2012.12'])

,L,2,3,4,H,Non R&D
1981.07 - 2012.12,1.090099,1.198580,1.252773,1.523258,2.287221,1.391910
1981.07 - 1999.12,1.286790,1.461857,1.543674,1.927867,2.695530,1.466495
2000.01 - 2012.12,0.810193,0.823915,0.838799,0.947467,1.706166,1.285770


# Task 2: Long-Short Portfolio

#### Merging Fama French Data with Returns Data

In [14]:
rd_reg = rd_81_12.copy()
rd_reg = pd.merge(rd_reg, fama, how = 'left', left_on= rd_reg.index, right_on = 'date')
rd_reg.head()

,1,2,3,4,5,date,Mkt-RF,SMB,HML,RF
0,0.086893,0.102657,0.074239,0.057083,0.072143,1981-07,-0.0154,-2.18,-0.49,0.0124
1,0.032224,0.027835,0.024060,0.035383,0.015164,1981-08,-0.0704,-1.94,4.79,0.0128
2,-0.017559,-0.015756,-0.010972,-0.012179,-0.014156,1981-09,-0.0717,-2.65,5.17,0.0124
3,-0.023296,-0.029053,-0.022388,-0.010710,0.011215,1981-10,0.0492,2.23,-4.21,0.0121
4,-0.038464,-0.050264,-0.033841,-0.037169,-0.036710,1981-11,0.0336,-1.03,1.85,0.0107


In [15]:
# SCL Regression
y = (-rd_reg[1] + rd_reg[5])/2
x = rd_reg['Mkt-RF'].copy().dropna()
x = sm.add_constant(x)
model = sm.OLS(y, x)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                   0.02828
Date:                Sun, 21 Nov 2021   Prob (F-statistic):              0.867
Time:                        18:37:16   Log-Likelihood:                 835.56
No. Observations:                 378   AIC:                            -1667.
Df Residuals:                     376   BIC:                            -1659.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0060      0.001      4.320      0.000       0.003       0.009
Mkt-RF         0.0050      0.030      0.168      0.867      -0.054       0.064
==============================================================================
Omnibus:                      235.377   Durbin-Watson:                   1.656
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2415.738
Skew:                           2.499   Prob(JB):                         0.00
Kurtosis:                      14.331   Cond. No.                         21.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
print(f"The SCL alpha is {result.params[0]*100}.")

The CAPM alpha is 0.5956873582970584.


In [17]:
#Fama French Regression
x = rd_reg[['Mkt-RF', 'SMB', 'HML']].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     4.324
Date:                Sun, 21 Nov 2021   Prob (F-statistic):            0.00516
Time:                        18:37:16   Log-Likelihood:                 841.99
No. Observations:                 378   AIC:                            -1676.
Df Residuals:                     374   BIC:                            -1660.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0067      0.001      4.838      0.000       0.004       0.009
Mkt-RF        -0.0062      0.031     -0.200      0.841      -0.067       0.055
SMB           -0.0012      0.000     -2.574      0.010      -0.002      -0.000
HML           -0.0015      0.000     -3.144      0.002      -0.003      -0.001
==============================================================================
Omnibus:                      234.226   Durbin-Watson:                   1.692
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2453.509
Skew:                           2.473   Prob(JB):                         0.00
Kurtosis:                      14.459   Cond. No.                         81.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
print(f"The Fama French Alpha is {results.params[0]*100}.")
print(f"The Sharpe Ratio is {y.mean() / y.std()}.")

The Fama French Alpha is 0.6667472236393671.
The Sharpe Ratio is 0.22529591446569466.


# Task 3: Repeat Steps 1 & 2 (Calculating Returns and Alphas) Using Value-Weighted Returns for the Portfolios

In [19]:
# Removing non-existent Market Caps
rd_df = rd_df.dropna(subset = ['me'])

# Find Total Market Cap by Date and Rank
market_caps = rd_df.groupby(['date', 'rank'])['me'].sum().reset_index()
rd_df = pd.merge(rd_df, market_caps, how = 'left', on = ['date', 'rank'])

# Find Weighted Returns for Each Date and Rank
rd_df['Weights'] = rd_df['me_x'] / rd_df['me_y']
rd_df['Weighted Returns'] = rd_df['Weights'] * rd_df['RET']

# Find Weighted Returns of Non-R&D Firms
market_caps_no_rd = no_rd.groupby('date')['me'].sum().reset_index()
no_rd2 = pd.merge(no_rd, market_caps_no_rd, how = 'left', on = 'date')
no_rd2['Weights'] = no_rd2['me_x'] / no_rd2['me_y']
no_rd2['Weighted Returns'] = no_rd2['Weights'] * no_rd2['RET']

In [20]:
# Calculating Returns from 1981 to 2012 for both R&D and Non-R&D Firms
rd_81_12 = rd_df[(rd_df['date'] >= '1981-07') & (rd_df['date'] <= '2012-12')].groupby(['date', 'rank'])['Weighted Returns'].sum().unstack(-1).dropna()
no_rd_81_12 = no_rd2[(no_rd2['date'] >= '1981-07') & (no_rd2['date'] <= '2012-12')].groupby(['date'])['Weighted Returns'].sum()

ret_81_12 = pd.merge(rd_81_12, no_rd_81_12, how = 'left', on = 'date')

# Calculating Returns from 1981.07 - 1999.12
rd_81_99 = rd_df[(rd_df['date'] >= '1981-07') & (rd_df['date'] <= '1999-12')].groupby(['date', 'rank'])['Weighted Returns'].sum().unstack(-1).dropna()
no_rd_81_99 = no_rd2[(no_rd2['date'] >= '1981-07') & (no_rd2['date'] <= '1999-12')].groupby(['date'])['Weighted Returns'].sum()

ret_81_99 = pd.merge(rd_81_99, no_rd_81_99, how = 'left', on = 'date')

# Calculating Returns from 2000.01 - 2012.12
rd_00_12 = rd_df[(rd_df['date'] >= '2000-01') & (rd_df['date'] <= '2012-12')].groupby(['date', 'rank'])['Weighted Returns'].sum().unstack(-1).dropna()
no_rd_00_12 = no_rd2[(no_rd2['date'] >= '2000-01') & (no_rd2['date'] <= '2012-12')].groupby(['date'])['Weighted Returns'].sum()

ret_00_12 = pd.merge(rd_00_12, no_rd_00_12, how = 'left', on = 'date')

In [21]:
pd.DataFrame(np.array([ret_81_12.mean(axis = 0) * 100, ret_81_99.mean(axis = 0) * 100, ret_00_12.mean(axis = 0) * 100]), columns = ['L', '2', '3', '4', 'H', 'Non R&D'], index = ['1981.07 - 2012.12', '1981.07 - 1999.12', '2000.01 - 2012.12'])

,L,2,3,4,H,Non R&D
1981.07 - 2012.12,0.979668,1.158433,1.038961,1.310794,1.173046,1.072982
1981.07 - 1999.12,1.353138,1.717972,1.536377,1.670423,1.695965,1.386507
2000.01 - 2012.12,0.448191,0.362167,0.331100,0.799014,0.428891,0.626811


In [22]:
# Merging Fama and Return Data
rd_reg = rd_81_12.copy()
rd_reg = pd.merge(rd_reg, fama, how = 'left', left_on= rd_reg.index, right_on = 'date')

In [23]:
# SCL Regression
y = (-rd_reg[1] + rd_reg[5])/2
x = rd_reg['Mkt-RF'].copy().dropna()
x = sm.add_constant(x)
model = sm.OLS(y, x)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                   0.05516
Date:                Sun, 21 Nov 2021   Prob (F-statistic):              0.814
Time:                        18:37:18   Log-Likelihood:                 860.68
No. Observations:                 378   AIC:                            -1717.
Df Residuals:                     376   BIC:                            -1709.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0009      0.001      0.720      0.472      -0.002       0.003
Mkt-RF         0.0066      0.028      0.235      0.814      -0.049       0.062
==============================================================================
Omnibus:                       73.313   Durbin-Watson:                   1.811
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              241.384
Skew:                           0.848   Prob(JB):                     3.84e-53
Kurtosis:                       6.529   Cond. No.                         21.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
print(f"The SCL alpha is {result.params[0] * 100}.")

The CAPM alpha is 0.09293401905942714.


In [25]:
# Fama French Regression
x = rd_reg[['Mkt-RF', 'SMB', 'HML']].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.2386
Date:                Sun, 21 Nov 2021   Prob (F-statistic):              0.869
Time:                        18:37:18   Log-Likelihood:                 861.02
No. Observations:                 378   AIC:                            -1714.
Df Residuals:                     374   BIC:                            -1698.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0011      0.001      0.834      0.405      -0.001       0.004
Mkt-RF         0.0032      0.030      0.109      0.913      -0.055       0.061
SMB           -0.0002      0.000     -0.498      0.619      -0.001       0.001
HML           -0.0004      0.000     -0.758      0.449      -0.001       0.001
==============================================================================
Omnibus:                       68.843   Durbin-Watson:                   1.813
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              211.815
Skew:                           0.816   Prob(JB):                     1.01e-46
Kurtosis:                       6.284   Cond. No.                         81.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
print(f"The Fama French alpha is {results.params[0] * 100}.")
print(f"The Sharpe Ratio is {y.mean() / y.std()}.")

The Fama French alpha is 0.10925180716420409.
The Sharpe Ratio is 0.038892656375066836.


# Task 4: Exclude the Largest 1000 Firms Each Year and Using Value-Weighted Returns

#### Creating a Table with the Companies

In [27]:
# Show largest companies each year
df2 = df.copy()
table = pd.crosstab(df2['year'], df2['LPERMNO'], values = df2['me'], aggfunc = sum)
companies = {}
for year in table.index:
    companies[year] = table.loc[year, :].sort_values(ascending = False).index[1000:]
    
# Convert companies dictionary into a df
companies = pd.DataFrame(companies)

# Create df to filter out 1000 largest companies
years = list(companies.keys())
filtered_data = pd.DataFrame()
for i in years:
    x = df2[(df2['year'] == i) & (df2['LPERMNO'].isin(list(companies[i])))]
    filtered_data = pd.concat([filtered_data, x])
    
# Create R&D and non-R&D dataframes
filtered_rd = filtered_data[filtered_data['xrd_me'] != 0]
filtered_no_rd = filtered_data[filtered_data['xrd_me'] == 0]

In [28]:
filtered_rd = pd.merge(filtered_rd, rd_df[['date', 'PERMNO', 'rank']].copy(), how = 'left', on = ['date', 'PERMNO'])

#### Equal Weighted Returns without Largest 1000

In [29]:
# Calculating Returns from 1981 to 2012 for both R&D and Non-R&D Firms
frd_81_12 = filtered_rd[(filtered_rd['date'] >= '1981-07') & (filtered_rd['date'] <= '2012-12')].groupby(['date', 'rank'])['RET'].mean().unstack(-1).dropna()
fno_rd_81_12 = filtered_no_rd[(filtered_no_rd['date'] >= '1981-07') & (filtered_no_rd['date'] <= '2012-12')].groupby(['date'])['RET'].mean()

fret_81_12 = pd.merge(frd_81_12, fno_rd_81_12, how = 'left', on = 'date')

# Calculating Returns from 1981.07 - 1999.12
frd_81_99 = filtered_rd[(filtered_rd['date'] >= '1981-07') & (filtered_rd['date'] <= '1999-12')].groupby(['date', 'rank'])['RET'].mean().unstack(-1).dropna()
fno_rd_81_99 = filtered_no_rd[(filtered_no_rd['date'] >= '1981-07') & (filtered_no_rd['date'] <= '1999-12')].groupby(['date'])['RET'].mean()

fret_81_99 = pd.merge(frd_81_99, fno_rd_81_99, how = 'left', on = 'date')

# Calculating Returns from 2000.01 - 2012.12
frd_00_12 = filtered_rd[(filtered_rd['date'] >= '2000-01') & (filtered_rd['date'] <= '2012-12')].groupby(['date', 'rank'])['RET'].mean().unstack(-1).dropna()
fno_rd_00_12 = filtered_no_rd[(filtered_no_rd['date'] >= '2000-01') & (filtered_no_rd['date'] <= '2012-12')].groupby(['date'])['RET'].mean()

fret_00_12 = pd.merge(frd_00_12, fno_rd_00_12, how = 'left', on = 'date')

In [30]:
pd.DataFrame(np.array([fret_81_12.mean(axis = 0) * 100, fret_81_99.mean(axis = 0) * 100, fret_00_12.mean(axis = 0) * 100]), columns = ['L', '2', '3', '4', 'H', 'Non R&D'], index = ['1981.07 - 2012.12', '1981.07 - 1999.12', '2000.01 - 2012.12'])

,L,2,3,4,H,Non R&D
1981.07 - 2012.12,1.024398,1.122757,1.188255,1.466501,2.318509,1.430247
1981.07 - 1999.12,1.229368,1.358049,1.503066,1.880704,2.742626,1.480305
2000.01 - 2012.12,0.732708,0.787918,0.740254,0.877057,1.714958,1.359009


#### Value Weighted Returns without Largest 1000

In [31]:
# Removing missing value Market Caps
filtered_rd = filtered_rd.dropna(subset = ['me'])

# Find Total ME by date and rank
market_caps2 = filtered_rd.groupby(['date', 'rank'])['me'].sum().reset_index()
filtered_rd = pd.merge(filtered_rd, market_caps2, how = 'left', on = ['date', 'rank'])

# Find Weighted Returns for date and rank
filtered_rd['Weights'] = filtered_rd['me_x'] / filtered_rd['me_y']
filtered_rd['Weighted Returns'] = filtered_rd['Weights'] * filtered_rd['RET']

# Find Weighted Returns of Non-R&D Firms
market_caps_no_rd2 = filtered_no_rd.groupby('date')['me'].sum().reset_index()
filtered_no_rd = pd.merge(filtered_no_rd, market_caps_no_rd2, how = 'left', on = 'date')
filtered_no_rd['Weights'] = filtered_no_rd['me_x'] / filtered_no_rd['me_y']
filtered_no_rd['Weighted Returns'] = filtered_no_rd['Weights'] * filtered_no_rd['RET']

In [32]:
# Calculating Returns from 1981 to 2012 for both R&D and Non-R&D Firms
frd_81_12 = filtered_rd[(filtered_rd['date'] >= '1981-07') & (filtered_rd['date'] <= '2012-12')].groupby(['date', 'rank'])['Weighted Returns'].sum().unstack(-1).dropna()
fno_rd_81_12 = filtered_no_rd[(filtered_no_rd['date'] >= '1981-07') & (filtered_no_rd['date'] <= '2012-12')].groupby(['date'])['Weighted Returns'].sum()

fret_81_12 = pd.merge(frd_81_12, fno_rd_81_12, how = 'left', on = 'date')

# Calculating Returns from 1981.07 - 1999.12
frd_81_99 = filtered_rd[(filtered_rd['date'] >= '1981-07') & (filtered_rd['date'] <= '1999-12')].groupby(['date', 'rank'])['Weighted Returns'].sum().unstack(-1).dropna()
fno_rd_81_99 = filtered_no_rd[(filtered_no_rd['date'] >= '1981-07') & (filtered_no_rd['date'] <= '1999-12')].groupby(['date'])['Weighted Returns'].sum()

fret_81_99 = pd.merge(frd_81_99, fno_rd_81_99, how = 'left', on = 'date')

# Calculating Returns from 2000.01 - 2012.12
frd_00_12 = filtered_rd[(filtered_rd['date'] >= '2000-01') & (filtered_rd['date'] <= '2012-12')].groupby(['date', 'rank'])['Weighted Returns'].sum().unstack(-1).dropna()
fno_rd_00_12 = filtered_no_rd[(filtered_no_rd['date'] >= '2000-01') & (filtered_no_rd['date'] <= '2012-12')].groupby(['date'])['Weighted Returns'].sum()

fret_00_12 = pd.merge(frd_00_12, fno_rd_00_12, how = 'left', on = 'date')

In [33]:
pd.DataFrame(np.array([fret_81_12.mean(axis = 0) * 100, fret_81_99.mean(axis = 0) * 100, fret_00_12.mean(axis = 0) * 100]), columns = ['L', '2', '3', '4', 'H', 'Non R&D'], index = ['1981.07 - 2012.12', '1981.07 - 1999.12', '2000.01 - 2012.12'])

,L,2,3,4,H,Non R&D
1981.07 - 2012.12,0.766019,0.811046,0.920247,1.076001,1.620758,1.014035
1981.07 - 1999.12,0.937926,1.102941,1.234189,1.379572,1.928230,1.060538
2000.01 - 2012.12,0.521383,0.395658,0.473482,0.643996,1.183202,0.947857


In [34]:
# SCL Regression on Value-weighted returns
frd_reg = frd_81_12.copy()
frd_reg = pd.merge(frd_reg, fama, how = 'left', left_on= frd_reg.index, right_on = 'date')

y = (-frd_reg[1] + frd_reg[5])/2
x = frd_reg['Mkt-RF'].copy().dropna()
x = sm.add_constant(x)
model = sm.OLS(y, x)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                   0.03062
Date:                Sun, 21 Nov 2021   Prob (F-statistic):              0.861
Time:                        18:37:24   Log-Likelihood:                 836.19
No. Observations:                 378   AIC:                            -1668.
Df Residuals:                     376   BIC:                            -1661.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0043      0.001      3.126      0.002       0.002       0.007
Mkt-RF        -0.0052      0.030     -0.175      0.861      -0.064       0.054
==============================================================================
Omnibus:                      168.363   Durbin-Watson:                   1.804
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1214.326
Skew:                           1.725   Prob(JB):                    2.05e-264
Kurtosis:                      11.075   Cond. No.                         21.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
print(f"The SCL alpha is {result.params[0] * 100}.")

The CAPM alpha is 0.4303544618127545.


In [36]:
# Fama French Regression on value-weighted returns
x = frd_reg[['Mkt-RF', 'SMB', 'HML']].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     3.820
Date:                Sun, 21 Nov 2021   Prob (F-statistic):             0.0102
Time:                        18:37:25   Log-Likelihood:                 841.88
No. Observations:                 378   AIC:                            -1676.
Df Residuals:                     374   BIC:                            -1660.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0049      0.001      3.558      0.000       0.002       0.008
Mkt-RF        -0.0110      0.031     -0.352      0.725      -0.072       0.050
SMB           -0.0013      0.000     -2.754      0.006      -0.002      -0.000
HML           -0.0013      0.000     -2.665      0.008      -0.002      -0.000
==============================================================================
Omnibus:                      172.860   Durbin-Watson:                   1.831
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1373.163
Skew:                           1.745   Prob(JB):                    6.63e-299
Kurtosis:                      11.660   Cond. No.                         81.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [37]:
print(f"The Fama French alpha is {results.params[0] * 100}.")
print(f"The Sharpe Ratio is {y.mean() / y.std()}.")

The Fama French alpha is 0.49048052311071483.
The Sharpe Ratio is 0.1611265253000265.


# Task 5: Repeat tasks for Extended Sample Period Through December 2020

In [38]:
# Calculating Returns from 1981 to 2020 for both R&D and Non-R&D Firms
rd_81_12 = rd_df[(rd_df['date'] >= '1981-07') & (rd_df['date'] <= '2020-12')].groupby(['date', 'rank'])['RET'].mean().unstack(-1).dropna()
no_rd_81_12 = no_rd[(no_rd['date'] >= '1981-07') & (no_rd['date'] <= '2020-12')].groupby(['date'])['RET'].mean()

ret_81_12 = pd.merge(rd_81_12, no_rd_81_12, how = 'left', on = 'date')

# Calculating Returns from 1981.07 - 1999.12
#Note I am adding 8 years for consistency as 8 years were added from 2012 for the above time period
rd_81_99 = rd_df[(rd_df['date'] >= '1981-07') & (rd_df['date'] <= '2007-12')].groupby(['date', 'rank'])['RET'].mean().unstack(-1).dropna()
no_rd_81_99 = no_rd[(no_rd['date'] >= '1981-07') & (no_rd['date'] <= '2007-12')].groupby(['date'])['RET'].mean()

ret_81_99 = pd.merge(rd_81_99, no_rd_81_99, how = 'left', on = 'date')

# Calculating Returns from 2000.01 - 2020.12

rd_00_12 = rd_df[(rd_df['date'] >= '2000-01') & (rd_df['date'] <= '2020-12')].groupby(['date', 'rank'])['RET'].mean().unstack(-1).dropna()
no_rd_00_12 = no_rd[(no_rd['date'] >= '2000-01') & (no_rd['date'] <= '2020-12')].groupby(['date'])['RET'].mean()

ret_00_12 = pd.merge(rd_00_12, no_rd_00_12, how = 'left', on = 'date')

#### Redoing Task 1 for New Time Period Equal Weighted Returns

In [39]:
pd.DataFrame(np.array([ret_81_12.mean(axis = 0) * 100, ret_81_99.mean(axis = 0) * 100, ret_00_12.mean(axis = 0) * 100]), columns = ['L', '2', '3', '4', 'H', 'Non R&D'], index = ['1981.07 - 2020.12', '1981.07 - 2007.12', '2000.01 - 2020.12'])

,L,2,3,4,H,Non R&D
1981.07 - 2020.12,1.125186,1.216103,1.304129,1.544383,2.203646,1.299357
1981.07 - 2007.12,1.115695,1.229376,1.268893,1.594788,2.345412,1.415775
2000.01 - 2020.12,0.982820,0.999605,1.093102,1.206551,1.770320,1.152115


#### Redoing Task 2

In [40]:
rd_reg = rd_81_12.copy()
rd_reg = pd.merge(rd_reg, fama, how = 'left', left_on= rd_reg.index, right_on = 'date')
rd_reg.head()

,1,2,3,4,5,date,Mkt-RF,SMB,HML,RF
0,0.086893,0.102657,0.074239,0.057083,0.072143,1981-07,-0.0154,-2.18,-0.49,0.0124
1,0.032224,0.027835,0.024060,0.035383,0.015164,1981-08,-0.0704,-1.94,4.79,0.0128
2,-0.017559,-0.015756,-0.010972,-0.012179,-0.014156,1981-09,-0.0717,-2.65,5.17,0.0124
3,-0.023296,-0.029053,-0.022388,-0.010710,0.011215,1981-10,0.0492,2.23,-4.21,0.0121
4,-0.038464,-0.050264,-0.033841,-0.037169,-0.036710,1981-11,0.0336,-1.03,1.85,0.0107


In [41]:
# New time period CAPM regression
y = (-rd_reg[1] + rd_reg[5])/2
x = rd_reg['Mkt-RF'].copy().dropna()
x = sm.add_constant(x)
model = sm.OLS(y, x)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.1001
Date:                Sun, 21 Nov 2021   Prob (F-statistic):              0.752
Time:                        18:37:26   Log-Likelihood:                 1058.6
No. Observations:                 474   AIC:                            -2113.
Df Residuals:                     472   BIC:                            -2105.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0055      0.001      4.511      0.000       0.003       0.008
Mkt-RF        -0.0084      0.027     -0.316      0.752      -0.061       0.044
==============================================================================
Omnibus:                      256.341   Durbin-Watson:                   1.681
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2508.530
Skew:                           2.147   Prob(JB):                         0.00
Kurtosis:                      13.420   Cond. No.                         22.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [42]:
# Regression alpha for new time period with equal weighting
print(f"The SCL alpha is {result.params[0]*100}.")

The CAPM alpha is 0.5451925311638695.


In [43]:
#Fama french regression with equal weighting for new time period
x = rd_reg[['Mkt-RF', 'SMB', 'HML']].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     5.059
Date:                Sun, 21 Nov 2021   Prob (F-statistic):            0.00186
Time:                        18:37:26   Log-Likelihood:                 1066.0
No. Observations:                 474   AIC:                            -2124.
Df Residuals:                     470   BIC:                            -2107.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0058      0.001      4.810      0.000       0.003       0.008
Mkt-RF        -0.0081      0.027     -0.296      0.768      -0.062       0.046
SMB           -0.0011      0.000     -2.779      0.006      -0.002      -0.000
HML           -0.0013      0.000     -3.220      0.001      -0.002      -0.001
==============================================================================
Omnibus:                      256.856   Durbin-Watson:                   1.709
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2534.464
Skew:                           2.149   Prob(JB):                         0.00
Kurtosis:                      13.481   Cond. No.                         77.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [44]:
# New time period fama french results with equal weighting
print(f"The Fama French Alpha is {results.params[0]*100}.")
print(f"The Sharpe Ratio is {y.mean() / y.std()}.")

The Fama French Alpha is 0.5757018673982571.
The Sharpe Ratio is 0.20768060953407383.


#### Redoing Task 3

In [45]:
# Calculating Returns from 1981 to 2020 for both R&D and Non-R&D Firms
rd_81_12 = rd_df[(rd_df['date'] >= '1981-07') & (rd_df['date'] <= '2020-12')].groupby(['date', 'rank'])['Weighted Returns'].sum().unstack(-1).dropna()
no_rd_81_12 = no_rd2[(no_rd2['date'] >= '1981-07') & (no_rd2['date'] <= '2020-12')].groupby(['date'])['Weighted Returns'].sum()

ret_81_12 = pd.merge(rd_81_12, no_rd_81_12, how = 'left', on = 'date')

# Calculating Returns from 1981.07 - 2007.12
rd_81_99 = rd_df[(rd_df['date'] >= '1981-07') & (rd_df['date'] <= '2007-12')].groupby(['date', 'rank'])['Weighted Returns'].sum().unstack(-1).dropna()
no_rd_81_99 = no_rd2[(no_rd2['date'] >= '1981-07') & (no_rd2['date'] <= '2007-12')].groupby(['date'])['Weighted Returns'].sum()

ret_81_99 = pd.merge(rd_81_99, no_rd_81_99, how = 'left', on = 'date')

# Calculating Returns from 2000.01 - 2020.12
rd_00_12 = rd_df[(rd_df['date'] >= '2000-01') & (rd_df['date'] <= '2020-12')].groupby(['date', 'rank'])['Weighted Returns'].sum().unstack(-1).dropna()
no_rd_00_12 = no_rd2[(no_rd2['date'] >= '2000-01') & (no_rd2['date'] <= '2020-12')].groupby(['date'])['Weighted Returns'].sum()

ret_00_12 = pd.merge(rd_00_12, no_rd_00_12, how = 'left', on = 'date')

In [46]:
#Value weighted returns for the new time period
pd.DataFrame(np.array([ret_81_12.mean(axis = 0) * 100, ret_81_99.mean(axis = 0) * 100, ret_00_12.mean(axis = 0) * 100]), columns = ['L', '2', '3', '4', 'H', 'Non R&D'], index = ['1981.07 - 2020.12', '1981.07 - 2007.12', '2000.01 - 2020.12'])

,L,2,3,4,H,Non R&D
1981.07 - 2020.12,0.928096,1.238292,1.125782,1.302453,1.168536,1.008457
1981.07 - 2007.12,1.092132,1.243221,1.120513,1.319655,1.261370,1.146350
2000.01 - 2020.12,0.553655,0.815716,0.764067,0.978289,0.703896,0.675412


In [47]:
rd_reg = rd_81_12.copy()
rd_reg = pd.merge(rd_reg, fama, how = 'left', left_on= rd_reg.index, right_on = 'date')

In [48]:
# Task 3 SCL Regression with new time period
y = (-rd_reg[1] + rd_reg[5])/2
x = rd_reg['Mkt-RF'].copy().dropna()
x = sm.add_constant(x)
model = sm.OLS(y, x)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                  0.007130
Date:                Sun, 21 Nov 2021   Prob (F-statistic):              0.933
Time:                        18:37:27   Log-Likelihood:                 1098.9
No. Observations:                 474   AIC:                            -2194.
Df Residuals:                     472   BIC:                            -2186.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0012      0.001      1.096      0.273      -0.001       0.003
Mkt-RF        -0.0021      0.024     -0.084      0.933      -0.050       0.046
==============================================================================
Omnibus:                       78.969   Durbin-Watson:                   1.838
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              273.244
Skew:                           0.730   Prob(JB):                     4.63e-60
Kurtosis:                       6.421   Cond. No.                         22.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [49]:
print(f"The SCL alpha is {result.params[0] * 100}.")

The CAPM alpha is 0.12168152022255403.


In [50]:
# Fama french regression with value weighted for new time period
x = rd_reg[['Mkt-RF', 'SMB', 'HML']].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.5256
Date:                Sun, 21 Nov 2021   Prob (F-statistic):              0.665
Time:                        18:37:27   Log-Likelihood:                 1099.7
No. Observations:                 474   AIC:                            -2191.
Df Residuals:                     470   BIC:                            -2175.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0013      0.001      1.169      0.243      -0.001       0.003
Mkt-RF        -0.0014      0.025     -0.056      0.956      -0.051       0.049
SMB           -0.0004      0.000     -0.948      0.343      -0.001       0.000
HML           -0.0004      0.000     -0.994      0.321      -0.001       0.000
==============================================================================
Omnibus:                       75.419   Durbin-Watson:                   1.838
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              243.310
Skew:                           0.718   Prob(JB):                     1.47e-53
Kurtosis:                       6.203   Cond. No.                         77.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [51]:
print(f"The Fama French alpha is {results.params[0] * 100}.")
print(f"The Sharpe Ratio is {y.mean() / y.std()}.")

The Fama French alpha is 0.13031399920417536.
The Sharpe Ratio is 0.05042232657318286.


#### Redoing Task 4 for the new time period

In [52]:
# Examine largest companies each year
df2 = df.copy()
table = pd.crosstab(df2['year'], df2['LPERMNO'], values = df2['me'], aggfunc = sum)
companies = {}
for year in table.index:
    companies[year] = table.loc[year, :].sort_values(ascending = False).index[1000:]
    
# Convert companies Dictionary into a df
companies = pd.DataFrame(companies)

# Create df to filter out 1000 largest companies
years = list(companies.keys())
filtered_data = pd.DataFrame()
for i in years:
    x = df2[(df2['year'] == i) & (df2['LPERMNO'].isin(list(companies[i])))]
    filtered_data = pd.concat([filtered_data, x])
    
# Create R&D and non-R&D dataframes off the filter
filtered_rd = filtered_data[filtered_data['xrd_me'] != 0]
filtered_no_rd = filtered_data[filtered_data['xrd_me'] == 0]

In [53]:
filtered_rd = pd.merge(filtered_rd, rd_df[['date', 'PERMNO', 'rank']].copy(), how = 'left', on = ['date', 'PERMNO'])

In [54]:
#Equal weighted returns for new time period without 1000 firms
# Calculating Returns from 1981 to 2020 for both R&D and Non-R&D Firms
frd_81_12 = filtered_rd[(filtered_rd['date'] >= '1981-07') & (filtered_rd['date'] <= '2020-12')].groupby(['date', 'rank'])['RET'].mean().unstack(-1).dropna()
fno_rd_81_12 = filtered_no_rd[(filtered_no_rd['date'] >= '1981-07') & (filtered_no_rd['date'] <= '2020-12')].groupby(['date'])['RET'].mean()

fret_81_12 = pd.merge(frd_81_12, fno_rd_81_12, how = 'left', on = 'date')

# Calculating Returns from 1981.07 - 2007.12

frd_81_99 = filtered_rd[(filtered_rd['date'] >= '1981-07') & (filtered_rd['date'] <= '2007-12')].groupby(['date', 'rank'])['RET'].mean().unstack(-1).dropna()
fno_rd_81_99 = filtered_no_rd[(filtered_no_rd['date'] >= '1981-07') & (filtered_no_rd['date'] <= '2007-12')].groupby(['date'])['RET'].mean()

fret_81_99 = pd.merge(frd_81_99, fno_rd_81_99, how = 'left', on = 'date')

# Calculating Returns from 2000.01 - 2020.12

frd_00_12 = filtered_rd[(filtered_rd['date'] >= '2000-01') & (filtered_rd['date'] <= '2020-12')].groupby(['date', 'rank'])['RET'].mean().unstack(-1).dropna()
fno_rd_00_12 = filtered_no_rd[(filtered_no_rd['date'] >= '2000-01') & (filtered_no_rd['date'] <= '2020-12')].groupby(['date'])['RET'].mean()

fret_00_12 = pd.merge(frd_00_12, fno_rd_00_12, how = 'left', on = 'date')

In [55]:
#Equal weighted returns for new time period without 1000 firms
pd.DataFrame(np.array([fret_81_12.mean(axis = 0) * 100, fret_81_99.mean(axis = 0) * 100, fret_00_12.mean(axis = 0) * 100]), columns = ['L', '2', '3', '4', 'H', 'Non R&D'], index = ['1981.07 - 2020.12', '1981.07 - 2007.12', '2000.01 - 2020.12'])

,L,2,3,4,H,Non R&D
1981.07 - 2020.12,1.037996,1.100533,1.211354,1.475111,2.221744,1.324392
1981.07 - 2007.12,1.029052,1.150847,1.204228,1.551669,2.384131,1.446474
2000.01 - 2020.12,0.869406,0.873674,0.954369,1.117803,1.762871,1.187039


In [56]:
#Value weighted returns without 1000 firms for extended time period
# Removing missing value Market Caps
filtered_rd = filtered_rd.dropna(subset = ['me'])

# Find Total ME by date and rank
market_caps2 = filtered_rd.groupby(['date', 'rank'])['me'].sum().reset_index()
filtered_rd = pd.merge(filtered_rd, market_caps2, how = 'left', on = ['date', 'rank'])

# Find Weighted Returns for Each date and rank
filtered_rd['Weights'] = filtered_rd['me_x'] / filtered_rd['me_y']
filtered_rd['Weighted Returns'] = filtered_rd['Weights'] * filtered_rd['RET']

# Find Weighted Returns of Non-R&D Firms
market_caps_no_rd2 = filtered_no_rd.groupby('date')['me'].sum().reset_index()
filtered_no_rd = pd.merge(filtered_no_rd, market_caps_no_rd2, how = 'left', on = 'date')
filtered_no_rd['Weights'] = filtered_no_rd['me_x'] / filtered_no_rd['me_y']
filtered_no_rd['Weighted Returns'] = filtered_no_rd['Weights'] * filtered_no_rd['RET']

In [57]:
# Calculating Returns from 1981 to 2020 for both R&D and Non-R&D Firms
frd_81_12 = filtered_rd[(filtered_rd['date'] >= '1981-07') & (filtered_rd['date'] <= '2020-12')].groupby(['date', 'rank'])['Weighted Returns'].sum().unstack(-1).dropna()
fno_rd_81_12 = filtered_no_rd[(filtered_no_rd['date'] >= '1981-07') & (filtered_no_rd['date'] <= '2020-12')].groupby(['date'])['Weighted Returns'].sum()

fret_81_12 = pd.merge(frd_81_12, fno_rd_81_12, how = 'left', on = 'date')

# Calculating Returns from 1981.07 - 2007.12
#going to 2007 to match the addition of 8 years to the above time period
frd_81_99 = filtered_rd[(filtered_rd['date'] >= '1981-07') & (filtered_rd['date'] <= '2007-12')].groupby(['date', 'rank'])['Weighted Returns'].sum().unstack(-1).dropna()
fno_rd_81_99 = filtered_no_rd[(filtered_no_rd['date'] >= '1981-07') & (filtered_no_rd['date'] <= '2007-12')].groupby(['date'])['Weighted Returns'].sum()

fret_81_99 = pd.merge(frd_81_99, fno_rd_81_99, how = 'left', on = 'date')

# Calculating Returns from 2000.01 - 2020.12
frd_00_12 = filtered_rd[(filtered_rd['date'] >= '2000-01') & (filtered_rd['date'] <= '2020-12')].groupby(['date', 'rank'])['Weighted Returns'].sum().unstack(-1).dropna()
fno_rd_00_12 = filtered_no_rd[(filtered_no_rd['date'] >= '2000-01') & (filtered_no_rd['date'] <= '2020-12')].groupby(['date'])['Weighted Returns'].sum()

fret_00_12 = pd.merge(frd_00_12, fno_rd_00_12, how = 'left', on = 'date')

In [58]:
#Value weighted without top 1000 firms for extended time period
pd.DataFrame(np.array([fret_81_12.mean(axis = 0) * 100, fret_81_99.mean(axis = 0) * 100, fret_00_12.mean(axis = 0) * 100]), columns = ['L', '2', '3', '4', 'H', 'Non R&D'], index = ['1981.07 - 2020.12', '1981.07 - 2007.12', '2000.01 - 2020.12'])

,L,2,3,4,H,Non R&D
1981.07 - 2020.12,0.738047,0.844696,0.995669,1.094564,1.663544,0.915251
1981.07 - 2007.12,0.784393,0.816013,0.906899,1.087294,1.670116,1.052934
2000.01 - 2020.12,0.561963,0.617194,0.785543,0.843485,1.430368,0.787259


In [59]:
# SCL regression for value weights for extended time period without top 1000 firms
frd_reg = frd_81_12.copy()
frd_reg = pd.merge(frd_reg, fama, how = 'left', left_on= frd_reg.index, right_on = 'date')

y = (-frd_reg[1] + frd_reg[5])/2
x = frd_reg['Mkt-RF'].copy().dropna()
x = sm.add_constant(x)
model = sm.OLS(y, x)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.1019
Date:                Sun, 21 Nov 2021   Prob (F-statistic):              0.750
Time:                        18:37:34   Log-Likelihood:                 1070.6
No. Observations:                 474   AIC:                            -2137.
Df Residuals:                     472   BIC:                            -2129.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0047      0.001      3.977      0.000       0.002       0.007
Mkt-RF        -0.0083      0.026     -0.319      0.750      -0.059       0.043
==============================================================================
Omnibus:                      192.956   Durbin-Watson:                   1.836
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1388.977
Skew:                           1.591   Prob(JB):                    2.44e-302
Kurtosis:                      10.759   Cond. No.                         22.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [60]:
print(f"The SCL alpha is {result.params[0] * 100}.")

The CAPM alpha is 0.46861378888259764.


In [61]:
# Fama french value weighted returns for extended time period without top 1000 firms
x = frd_reg[['Mkt-RF', 'SMB', 'HML']].copy().dropna()
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     4.262
Date:                Sun, 21 Nov 2021   Prob (F-statistic):            0.00551
Time:                        18:37:34   Log-Likelihood:                 1076.9
No. Observations:                 474   AIC:                            -2146.
Df Residuals:                     470   BIC:                            -2129.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0049      0.001      4.202      0.000       0.003       0.007
Mkt-RF        -0.0037      0.027     -0.138      0.890      -0.056       0.049
SMB           -0.0012      0.000     -2.919      0.004      -0.002      -0.000
HML           -0.0010      0.000     -2.591      0.010      -0.002      -0.000
==============================================================================
Omnibus:                      198.488   Durbin-Watson:                   1.851
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1547.932
Skew:                           1.618   Prob(JB):                         0.00
Kurtosis:                      11.241   Cond. No.                         77.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [62]:
print(f"The Fama French alpha is {results.params[0] * 100}.")
print(f"The Sharpe Ratio is {y.mean() / y.std()}.")

The Fama French alpha is 0.49148176379784064.
The Sharpe Ratio is 0.18281157521129834.
